Needs fixing:
    
    -ineffcieint tf.diag .... we are doing n^2 computations when we could be being only n...figure out how to avoid

In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops
import networkx as nx
import numpy as np
import pandas as pd

communities = 2
group_size = 10

def Diffusion(A, F):
    """finds the diffusion of F via A"""
    #F can be a batched signal
    return tf.batch_matmul(A, F)


def Diag(A, F):
    """multiplies F by diagonal vector"""
    diag_matrix = tf.expand_dims(tf.reduce_sum(Adj, 1), 1)
    return tf.mul(diag_matrix, F)

def Diffusion_normed(A, F):
    """D^-1W"""
    return tf.mul(tf.div(1.0, Diag(A, F)), tf.batch_matmul(A, F))

def Laplacian(A, F):
    """find the laplacian of a graph A given signal F"""
    
    return tf.sub(tf.transpose(Diffusion(A,F)), Diag(A,F))

def Max_norm(F):
    """Returns the maximum of the signal"""
    return tf.reduce_max(F)

def L2_norm(F):
    """Returns L2 norm of F"""
    return tf.sqrt(tf.reduce_sum(tf.square(F)))

def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    """gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size"""

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


In [2]:
def batch_vm(v, m):
    shape = tf.shape(v)
    rank = shape.get_shape()[0].value
    v = tf.expand_dims(v, rank)

    vm = tf.mul(v, m)

    return tf.reduce_sum(vm, rank-1)

def batch_vm2(x, m):
    [input_size, output_size] = m.get_shape().as_list()

    input_shape = tf.shape(x)
    batch_rank = input_shape.get_shape()[0].value - 1
    batch_shape = input_shape[:batch_rank]
    output_shape = tf.concat(0, [batch_shape, [output_size]])

    x = tf.reshape(x, [-1, input_size])
    y = tf.matmul(x, m)

    y = tf.reshape(y, output_shape)

    return y

In [3]:
signal_dim = 3
communities = 2 #number of communities, chance to 
group_size = 2
Size = 100
p_min = 0.5
p_max = 0.6


#A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.6, p_out=0.01)
#Adj = tf.cast(A, dtype = tf.float32)
DATA = [np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, 0.1*p)).astype(np.double) for p in np.linspace(p_min, p_max, Size)]
DATA[1:2]
tf.__version__

'0.10.0'

In [403]:
#testing functions defined above


signal_dim = 2
batch_size = 3
communities = 2 #number of communities, chance to 
group_size = 2
dim = communities*group_size
Size = 100
p_min = 0.8
p_max = 0.9
Mean = 0.5


#A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.6, p_out=0.01)
#Adj = tf.cast(A, dtype = tf.float32)
DATA = [np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, 0.1*p)).astype(np.double) for p in np.linspace(p_min, p_max, Size)]


Adj = tf.placeholder(dtype=tf.float32, shape=[None, communities*group_size, communities*group_size])
Adj_mod = tf.reshape(tf.transpose(Adj, perm = [1,0,2]), [dim, batch_size*dim])#preparing it to be multiplied by F to broadcast

#initialize a 10 dim signal
F = tf.transpose(tf.Variable(tf.random_normal([communities*group_size, signal_dim], stddev=1.0, mean=0.0, seed=1)))
 
#first diffusion step without cascading (unnormed version)
Diff_1 = tf.reshape(tf.transpose(tf.matmul(F, Adj_mod)), shape=[batch_size, dim, signal_dim]) #shape=[batch_size, signal_dim, dim])

diag_inv = tf.div(1.0, tf.reduce_sum(Adj, 2))
diag_inv_batch = tf.matrix_diag(diag_inv) #to use in subsequent layers
Diag_1 = tf.mul(tf.expand_dims(diag_inv, 1), F)


C_a = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

#treat this as the new Adj_mod
A1 = tf.matmul(C_a, tf.reshape(tf.transpose(Diag_1, perm=[1, 0,2]), [signal_dim, batch_size*dim]))
B1 = tf.matmul(C_b, tf.reshape(tf.transpose(Diff_1, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

#transform it back into the 3-D tensor it is
#Psi_1 = tf.transpose(tf.reshape(A1 + B1, shape = [signal_dim, batch_size, dim]), perm=[1,0,2])
#relu also added
Psi_1 = tf.transpose(tf.reshape(tf.nn.relu(A1 + B1), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])

###################
###################
Diff_2 = tf.batch_matmul(Adj, Psi_1)
Diag_2 = tf.batch_matmul(diag_inv_batch, Psi_1)
#we change the constants for now but let's keep these the same for another model
C_a_1 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b_1 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

A2 = tf.matmul(C_a_1, tf.reshape(tf.transpose(Diag_2, perm=[2, 0,1]), [signal_dim, batch_size*dim]))
B2 = tf.matmul(C_b_1, tf.reshape(tf.transpose(Diff_2, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

Psi_2 = tf.transpose(tf.reshape(tf.nn.relu(A2 + B2), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])

##################
##################
Diff_3 = tf.batch_matmul(Adj, Psi_2)
Diag_3 = tf.batch_matmul(diag_inv_batch, Psi_2)

C_a_2 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b_2 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

A3 = tf.matmul(C_a_2, tf.reshape(tf.transpose(Diag_3, perm=[2, 0,1]), [signal_dim, batch_size*dim]))
B3 = tf.matmul(C_b_2, tf.reshape(tf.transpose(Diff_3, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

Psi_3 = tf.transpose(tf.reshape(tf.nn.relu(A3 + B3), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])

##################
##################
Diff_4 = tf.batch_matmul(Adj, Psi_3)
Diag_4 = tf.batch_matmul(diag_inv_batch, Psi_3)

C_a_3 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b_3 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

A4 = tf.matmul(C_a_3, tf.reshape(tf.transpose(Diag_4, perm=[2, 0,1]), [signal_dim, batch_size*dim]))
B4 = tf.matmul(C_b_3, tf.reshape(tf.transpose(Diff_4, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

Psi_4 = tf.transpose(tf.reshape(tf.nn.relu(A4 + B4), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])


##################
##################
Diff_5 = tf.batch_matmul(Adj, Psi_4)
Diag_5 = tf.batch_matmul(diag_inv_batch, Psi_4)

C_a_4 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b_4 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

A5 = tf.matmul(C_a_4, tf.reshape(tf.transpose(Diag_5, perm=[2, 0,1]), [signal_dim, batch_size*dim]))
B5 = tf.matmul(C_b_4, tf.reshape(tf.transpose(Diff_5, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

Psi_5 = tf.transpose(tf.reshape(tf.nn.relu(A5 + B5), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])


##################
##################
Diff_6 = tf.batch_matmul(Adj, Psi_5)
Diag_6 = tf.batch_matmul(diag_inv_batch, Psi_5)

C_a_5 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))
C_b_5 = tf.Variable(tf.random_normal([signal_dim, signal_dim], stddev=1.0, mean=Mean))

A6 = tf.matmul(C_a_5, tf.reshape(tf.transpose(Diag_6, perm=[2, 0,1]), [signal_dim, batch_size*dim]))
B6 = tf.matmul(C_b_5, tf.reshape(tf.transpose(Diff_6, perm=[2, 0,1]), [signal_dim, batch_size*dim]))

Psi_6 = tf.transpose(tf.reshape(tf.nn.relu(A6 + B6), shape = [signal_dim, batch_size, dim]), perm=[1,2,0])

##################
##################

init = tf.initialize_all_variables()

with tf.Session() as sess:
    for i in xrange(1):
        sess.run(init)
        
        print sess.run([Psi_6], feed_dict={Adj: DATA[i:i+batch_size]})

        #print sess.run([tf.batch_matmul(tf.matrix_diag(diag_inv), Psi_1)], feed_dict={Adj: DATA[i:i+batch_size]})

        #print sess.run([tf.transpose(Psi_1, perm=[1, 0, 2])], feed_dict={Adj: DATA[i:i+batch_size]})
        #print sess.run([tf.reshape(A1+B1, shape=[2, batch_size, dim])], feed_dict={Adj: DATA[i:i+batch_size]})
        #print sess.run([tf.shape(Diag_mod)], feed_dict={Adj: DATA[i:i+batch_size]})
        #print sess.run([B1], feed_dict={Adj: DATA[i:i+batch_size]})

        

[array([[[   0.        ,    0.        ],
        [   0.        ,    0.        ],
        [ 192.90757751,  131.36637878],
        [ 202.95181274,  134.9052124 ]],

       [[ 201.71510315,  135.57440186],
        [ 212.63111877,  139.18081665],
        [ 192.90757751,  131.36637878],
        [ 202.95181274,  134.9052124 ]],

       [[ 201.71510315,  135.57440186],
        [ 212.63111877,  139.18081665],
        [ 192.90757751,  131.36637878],
        [ 202.95181274,  134.9052124 ]]], dtype=float32)]


In [388]:
#tf.reset_default_graph()
np.dot(np.asarray([ 0.07222086,  4.90196609]),np.asarray([-1.18431401,  1.38316786]))


6.694709770185618